In [51]:
# #빅데이터_수집_저장 #미니실습,
# 정적 크롤링,
# 관심 있는 사이트에가서,
#
# 1) 해당 사이트 주소/robots.txt
# 확인해서, 허용 가능한지 여부 판단.
#
# 2) 해당 사이트 구조,
# 리액트 처럼, 특정의 함수등을 호출해서,
# 데이터를 받아서 출력이면,
# 수집 할 데이터가, 화면에서, 페이지 소스 보기를 했을 경우,
# 해당 데이터가 정적으로 존재해야 수집 가능.
#
# 3) 3가지 이상의 데이터를 수집
# -(메모리)
# - json
# - csv
# - 데이터베이스 저장.

In [74]:
import re

import bs4
import urllib.request
import csv
import json
import pymysql
wikipediaUrl = "https://en.wikipedia.org/wiki/78th_British_Academy_Film_Awards"
req = urllib.request.Request(wikipediaUrl, headers={"User-Agent": "Mozilla/5.0"})
htmlObject = urllib.request.urlopen(wikipediaUrl)
webPage = htmlObject.read()
bsObject = bs4.BeautifulSoup(webPage, 'html.parser')

#전체 페이지 텍스트 저장
full_text = bsObject.get_text(separator="\n",strip=True)

#전체 텍스트를 파일로 저장
text_filename = "wikipedia_full_text.txt"
with open(text_filename, "w", encoding="utf-8") as text_file:
    text_file.write(full_text)

print(f"📄 전체 페이지 텍스트가 {text_filename}에 저장되었습니다.")

#페이지 제목 추출
page_title = bsObject.find("h1", {"id": "firstHeading"}).text.strip()
print("page_title:", page_title)

#위키테이블 데이터 추출
wikitable_list = bsObject.find_all("table", {"class": "wikitable"})
print(f"Found {len(wikitable_list)} wikitable(s)")

📄 전체 페이지 텍스트가 wikipedia_full_text.txt에 저장되었습니다.
page_title: 78th British Academy Film Awards
Found 3 wikitable(s)


In [76]:
#결과를 저장할 딕셔너리(JSON 저장용)
tables_data ={}

#CSV, JSON, DB에 저장을 위한 각 테이블 처리
for table_index, table in enumerate(wikitable_list, start=1):
    table_data = []
    rows = table.find_all("tr")

    for row in rows:
        cells = row.find_all(["th", "td"])
        row_data = [cell.get_text(strip=True) for cell in cells]
        table_data.append(row_data)

    #CSV 파일로 저장(테이블 번호별 파일 생성)
    csv_filename = f"wikitable_{table_index}.csv"
    with open(csv_filename, "w", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(table_data)
    print(f"📁 Table {table_index} 데이터가 {csv_filename}에 저장되었습니다.")

    #JSON 저장용 딕셔너리에 추가
    tables_data[f"table_{table_index}"] = table_data

#JSON 파일에 저장(모든 테이블 데이터)
json_filename = "wikitable_data.json"
with open(json_filename, "w", encoding="utf-8") as json_file:
    json.dump(tables_data, json_file, ensure_ascii=False, indent=4)
print(f"📂 모든 테이블 데이터가 {json_filename}에 저장되었습니다.")

📁 Table 1 데이터가 wikitable_1.csv에 저장되었습니다.
📁 Table 2 데이터가 wikitable_2.csv에 저장되었습니다.
📁 Table 3 데이터가 wikitable_3.csv에 저장되었습니다.
📂 모든 테이블 데이터가 wikitable_data.json에 저장되었습니다.


In [77]:
# db 연결 작업,
db = pymysql.connect(host="localhost", user="webuser", passwd="webuser", db="webdb",charset="utf8mb4")
cursor = db.cursor()

def is_safe_name(name):
    return bool(re.match(r'^[A-Za-z0-9_]+$', name))

try:
    # 테이블 별 데이터 저장
    for table_name, table_data in tables_data.items():
        if not table_data:
            continue

        if not is_safe_name(table_name):
            print(f"테이블 이름 '{table_name}'이 유효하지 않습니다.")
            continue

        header = table_data[0]
        num_columns = len(header)

        for column in header:
            if not is_safe_name(column):
                print(f"컬럼 이름 '{column}'이 유효하지 않습니다.")
                continue

    # 7. 변경 사항 저장 (commit)
    db.commit()
    print("✅ MySQL 데이터베이스에 데이터 저장 완료!")

except Exception as e:
    db.rollback()  # 오류 발생 시 롤백
    print(f"❌ DB 저장 실패: {e}")

finally:
    cursor.close()
    db.close()  # 연결 종료


컬럼 이름 'Best FilmConclave–Tessa Ross, Juliette Howell, and Michael A. JackmanAnora– Alex Coco,Samantha Quan, andSean BakerThe Brutalist– Nick Gordon, Brian Young, Andrew Morrison, D.J. Gugenheim, andBrady CorbetA Complete Unknown– Alex Heineman,Fred Berger, andJames MangoldEmilia Pérez– Pascal Caucheteux andJacques Audiard'이 유효하지 않습니다.
컬럼 이름 'Best DirectorBrady Corbet–The BrutalistJacques Audiard–Emilia PérezSean Baker–AnoraEdward Berger–ConclaveCoralie Fargeat–The SubstanceDenis Villeneuve–Dune: Part Two'이 유효하지 않습니다.
✅ MySQL 데이터베이스에 데이터 저장 완료!
